In [1]:
import matplotlib
%matplotlib inline
from collections import defaultdict
import fiona
from fiona.crs import from_epsg
import matplotlib
import pyproj
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors
from ipywidgets import interact

proj_epsg_4326 = pyproj.Proj("+init=EPSG:4326")
proj_epsg_lv03 = pyproj.Proj("+init=EPSG:21781")


In [2]:
def create_graph():
    """
        Create a networkx Graph from the SBB "linie-mit-betriebspunkten"
        dataset.
    """
    all_stops = set()
    lines = defaultdict(list)

    with fiona.open("linie-mit-betriebspunkten.geojson") as src:
        for f in src:
            line = f['properties']['linie']
            stop = f['properties']['abkurzung_bpk']
            km = f['properties']['km']
            lonlat = f['geometry']['coordinates']

            x, y = pyproj.transform(proj_epsg_4326,
                                    proj_epsg_lv03,
                                    lonlat[0],
                                    lonlat[1])

            all_stops.add((stop, (x, y)))

            lines[line].append({'stop': stop, 'km': km, 'xy': (x, y)})

    # Sort lines
    for line in lines:
        lines[line] = sorted(lines[line], key=lambda x: x['km'])

    # Create graph
    G = nx.Graph(crs=from_epsg(4326), name="SBB")

    # Add vertex
    for stop in all_stops:
        data = {'x': stop[1][0],
                'y': stop[1][1],
                'coords': stop[1]}
        G.add_node(stop[0], pos=stop[1], **data)

    # Add edges
    for line in lines:
        for i, stop in enumerate(lines[line][:-1]):
            start = lines[line][i]['stop']
            end = lines[line][i + 1]['stop']
            length = lines[line][i + 1]['km'] - lines[line][i]['km']
            G.add_edge(start, end, length=length, name="_")

    return G

G = create_graph()

In [3]:
def plot_vertices(G, centrality_func, ax, node_size=20, min_degree=4):
    # Vertices centrality

    measures = centrality_func(G)

    # Select only nodes with degree >= min_degree
    nodelist = []
    for node in G.degree():
        if node[1] >= min_degree:
            nodelist.append(node[0])

    node_values = [measures[node] for node in nodelist]
    pos = nx.get_node_attributes(G, 'pos')

    # color scale converted to list of colors for graph edges
    norm = colors.Normalize(vmin=min(node_values) * 0.8,
                            vmax=max(node_values))

    cmap = cm.ScalarMappable(norm=norm,
                             cmap=cm.inferno)

    nc = [cmap.to_rgba(cl) for cl in node_values]

    nodes = nx.draw_networkx_nodes(G,
                                   pos=pos,
                                   arrows=False,
                                   with_labels=False,
                                   node_size=node_size,
                                   node_color=nc,
                                   nodelist=nodelist,
                                   ax=ax)

    edges = nx.draw_networkx_edges(G,
                                   pos,
                                   edge_color='#555555',
                                   alpha=1.0,
                                   width=1.0,
                                   ax=ax)

    ax.set_title("Vertices")
    ax.axis('off')
    ax.axis('equal')

In [4]:
def plot_edges(G, centrality_func, ax):

    L = nx.line_graph(G)

    measures = centrality_func(L)

    edgelist = list(G.edges())
    edge_values = [measures[tuple(sorted(edge))] for edge in edgelist]

    # color scale converted to list of colors for graph edges
    norm = colors.Normalize(vmin=min(edge_values) * 0.8,
                            vmax=max(edge_values))

    cmap = cm.ScalarMappable(norm=norm,
                             cmap=cm.inferno)
    ec = [cmap.to_rgba(cl) for cl in edge_values]

    pos = nx.get_node_attributes(G, 'pos')

    plt.style.use('dark_background')
    cmap = plt.cm.plasma

    edges = nx.draw_networkx_edges(G,
                                   pos,
                                   edgelist=edgelist,
                                   edge_color=ec,
                                   alpha=1.,
                                   node_size=1,
                                   edge_cmap=cmap,
                                   width=1,
                                   ax=ax)

    ax.set_title("Edges")
    ax.axis('off')
    ax.axis('equal')


In [5]:
def plot_centrality(centrality_func):

    if centrality_func == "degree":
        centrality_func = nx.degree_centrality
    elif centrality_func == "betweenness":
        centrality_func = nx.betweenness_centrality
    elif centrality_func == "pagerank":
        centrality_func = nx.pagerank
    elif centrality_func == "eigenvector":
        centrality_func = nx.eigenvector_centrality
    elif centrality_func == "katz":
        centrality_func = nx.katz_centrality
    elif centrality_func == "closeness":
        centrality_func = nx.closeness_centrality
    elif centrality_func == "harmonic":
        centrality_func = nx.harmonic_centrality
    
    plt.style.use('dark_background')

    fig, (ax1, ax2) = plt.subplots(2,
                                   1,
                                   figsize=(7, 10),
                                   dpi=600)


    print(centrality_func.__name__)

    fig.suptitle("Centrality measure: {}".format(centrality_func.__name__))

    plot_vertices(G, centrality_func, ax1)

    plot_edges(G, centrality_func, ax2)

    plt.show()

    
interact(plot_centrality, centrality_func=["degree", "betweenness", "pagerank", "eigenvector", "katz", "closeness", "harmonic"])

interactive(children=(Dropdown(description='centrality_func', options=('degree', 'betweenness', 'pagerank', 'e…

<function __main__.plot_centrality(centrality_func)>